# IMPORT

In [1]:
import pandas as pd
import numpy as np

from ipywidgets import widgets

from pandas_profiling import ProfileReport

import sweetviz as sv

# DEFINE

In [2]:
DATA_PATH = '../data/'

# PREPARATION

In [3]:
# train
train = pd.read_csv(DATA_PATH+'/train_clean.csv',
                    dtype={'floors': str,
                           'total_floors': str})
# test
test = pd.read_csv(DATA_PATH+'test_clean.csv',
                    dtype={'floors': str,
                           'total_floors': str})

In [4]:
new_features = pd.read_csv(DATA_PATH+'districts_info.csv',
                           dtype={'num_of_metro_stations': str,
                                  'num_of_kindg': str,
                                  'num_of_schools': str,
                                  'num_of_poly': str,
                                  'num_of_hospitals': str,
                                  'num_of_dentists': str,
                                  'num_of_women_cons': str,})

In [5]:
new_features.rename(columns={'Unnamed: 0':'district'}, inplace=True)

In [6]:
new_features

,district,num_of_metro_stations,num_of_kindg,num_of_schools,num_of_poly,num_of_hospitals,num_of_dentists,num_of_women_cons
0,Адмиралтейский,9,73,46,12,6,5,4
1,Ваcилеостровский,3,64,42,9,6,4,1
2,Выборгский,6,136,73,27,7,4,5
3,Калининский,5,113,61,24,3,3,4
4,Кировский,5,97,58,16,1,6,3
5,Колпинский,0,56,30,16,2,2,2
6,Красногвардейский,2,88,54,16,2,2,4
7,Красносельский,0,61,45,19,2,4,4
8,Московский,6,13,47,15,2,5,3
9,Невский,7,132,58,23,1,3,6


In [7]:
new_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   district               17 non-null     object
 1   num_of_metro_stations  17 non-null     object
 2   num_of_kindg           17 non-null     object
 3   num_of_schools         17 non-null     object
 4   num_of_poly            17 non-null     object
 5   num_of_hospitals       17 non-null     object
 6   num_of_dentists        17 non-null     object
 7   num_of_women_cons      17 non-null     object
dtypes: object(8)
memory usage: 1.2+ KB


# ADD NEW FEATURES

In [8]:
train = train.merge(new_features, how='left')

In [9]:
test = test.merge(new_features, how='left')

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98459 entries, 0 to 98458
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   page                   98459 non-null  object 
 1   description            98459 non-null  object 
 2   flat_type              98459 non-null  object 
 3   object_type            98459 non-null  object 
 4   rooms                  98459 non-null  object 
 5   floors                 98459 non-null  object 
 6   square                 98459 non-null  float64
 7   kitchen_square         55863 non-null  float64
 8   live_square            61432 non-null  float64
 9   price                  98459 non-null  float64
 10  build_matireal         98459 non-null  object 
 11  public_date            98459 non-null  object 
 12  update_date            98459 non-null  object 
 13  district_rating        98459 non-null  object 
 14  district               98459 non-null  object 
 15  un

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42192 entries, 0 to 42191
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   page                   42192 non-null  object 
 1   description            42192 non-null  object 
 2   flat_type              42192 non-null  object 
 3   object_type            42192 non-null  object 
 4   rooms                  42192 non-null  object 
 5   floors                 42192 non-null  object 
 6   square                 42192 non-null  float64
 7   kitchen_square         23937 non-null  float64
 8   live_square            26304 non-null  float64
 9   price                  42192 non-null  float64
 10  build_matireal         42192 non-null  object 
 11  public_date            42192 non-null  object 
 12  update_date            42192 non-null  object 
 13  district_rating        42192 non-null  object 
 14  district               42192 non-null  object 
 15  un

# Fill NaN

**В новых признаках есть пропуски, так как есть объявления в которых район указан как "Санкт-Петербург". Это объявления, которые находятся в лен. области. Узнать к какой конкретно району лен. области они относятся нет возможности, пока просто добавлю категорию "лен. обл.". При оценке модели нужно будет внимательней посмотреть на эти объявления, возможно в них модель будет ошибться чаще.**

In [12]:
def fill_new_feat_nan(df):

    temp_df = df.copy()

    for col in ['num_of_metro_stations', 'num_of_kindg', 'num_of_schools',
                'num_of_poly', 'num_of_hospitals', 'num_of_dentists', 'num_of_women_cons']:
        
        temp_df[col].fillna('Лен. область', inplace=True)
    
    return temp_df
    

In [13]:
train = fill_new_feat_nan(train)

In [14]:
test = fill_new_feat_nan(test)

In [15]:
train.drop(columns=['public_date', 'update_date'], inplace=True)

In [16]:
test.drop(columns=['public_date', 'update_date'], inplace=True)

In [17]:
train.to_csv('../data/train_new_features.csv', index=False)

In [18]:
test.to_csv('../data/test_new_features.csv', index=False)

# PANDAS PROFILING

In [19]:
df = train.append(test, sort=False).reset_index(drop=True)
df.drop(columns=['page'], inplace=True)

In [20]:
# Удаляем все выше перечисленные категории.
# flat_type - Своб. планировка.
df_correct = df.drop(df[df['flat_type'] == 'Своб. планировка'].index)
# rooms - Своб. планировка, 7, 8, 9.
df_correct.drop(df_correct[(df_correct['rooms'] == 'Своб. планировка') | (df_correct['rooms'] == '7') | \
                   (df_correct['rooms'] == '8') | (df_correct['rooms'] == '9')].index, inplace=True)
# floors - 26 - 35.
df_correct.drop(df_correct[(df_correct['floors'] == '26') | (df_correct['floors'] == '27') | \
                   (df_correct['floors'] == '28') | (df_correct['floors'] == '29') | \
                   (df_correct['floors'] == '30') | (df_correct['floors'] == '31') | \
                   (df_correct['floors'] == '32') | (df_correct['floors'] == '33') | \
                   (df_correct['floors'] == '34') | (df_correct['floors'] == '35')].index, inplace=True)
# build_matireal - блоки, дерево.
df_correct.drop(df_correct[(df_correct['build_matireal'] == 'Блоки') | \
                           (df_correct['build_matireal'] == 'Дерево')].index, inplace=True)
# total_floors - 1, 30 - 36.
df_correct.drop(df_correct[(df_correct['total_floors'] == '1') | (df_correct['total_floors'] == '30') | \
                   (df_correct['total_floors'] == '31') | (df_correct['total_floors'] == '32') | \
                   (df_correct['total_floors'] == '33') | (df_correct['total_floors'] == '34') | \
                   (df_correct['total_floors'] == '35') | (df_correct['total_floors'] == '36')].index, inplace=True)
# metro_station - 5км, Пискаревка.
df_correct.drop(df_correct[(df_correct['metro_station'] == '5 км') | \
                           (df_correct['metro_station'] == 'Пискарёвка')].index, inplace=True)

## < 400 m2

In [21]:
df_correct_400m2 = df_correct[(df_correct['square']<401) & (df_correct['price']<100000000)].reset_index(drop=True)

In [22]:
profile_cat_correct_400m2 = ProfileReport(df_correct_400m2, title="Correct categorical flats < 400m2")

In [23]:
# Сохраняем рузльтат в HTML.
profile_cat_correct_400m2.to_file("NEW_FEATURES_400M2_PANDAS_PROFILING_REPORT.html")

Summarize dataset:   0%|          | 0/45 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## < 130 m2

In [24]:
df_correct_130m2 = df_correct[(df_correct['square']<131) & (df_correct['price']<100000000)].reset_index(drop=True)

In [25]:
profile_cat_correct_130m2 = ProfileReport(df_correct_130m2, title="Correct categorical flats < 130m2")

In [26]:
# Сохраняем рузльтат в HTML.
profile_cat_correct_130m2.to_file("NEW_FEATURES_130M2_PANDAS_PROFILING_REPORT.html")

Summarize dataset:   0%|          | 0/45 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# SWEETVIZ

In [27]:
# Удаляем все выше перечисленные категории.
# flat_type - Своб. планировка.
train_correct = train.drop(train[train['flat_type'] == 'Своб. планировка'].index)
# rooms - Своб. планировка, 7, 8, 9.
train_correct.drop(train_correct[(train_correct['rooms'] == 'Своб. планировка') | (train_correct['rooms'] == '7') | \
                   (train_correct['rooms'] == '8') | (train_correct['rooms'] == '9')].index, inplace=True)
# floors - 26 - 35.
train_correct.drop(train_correct[(train_correct['floors'] == '26') | (train_correct['floors'] == '27') | \
                   (train_correct['floors'] == '28') | (train_correct['floors'] == '29') | \
                   (train_correct['floors'] == '30') | (train_correct['floors'] == '31') | \
                   (train_correct['floors'] == '32') | (train_correct['floors'] == '33') | \
                   (train_correct['floors'] == '34') | (train_correct['floors'] == '35')].index, inplace=True)
# build_matireal - блоки, дерево.
train_correct.drop(train_correct[(train_correct['build_matireal'] == 'Блоки') | \
                           (train_correct['build_matireal'] == 'Дерево')].index, inplace=True)
# total_floors - 1, 30 - 36.
train_correct.drop(train_correct[(train_correct['total_floors'] == '1') | (train_correct['total_floors'] == '30') | \
                   (train_correct['total_floors'] == '31') | (train_correct['total_floors'] == '32') | \
                   (train_correct['total_floors'] == '33') | (train_correct['total_floors'] == '34') | \
                   (train_correct['total_floors'] == '35') | (train_correct['total_floors'] == '36')].index, inplace=True)
# metro_station - 5км, Пискаревка.
train_correct.drop(train_correct[(train_correct['metro_station'] == '5 км') | \
                           (train_correct['metro_station'] == 'Пискарёвка')].index, inplace=True)

In [28]:
# Удаляем все выше перечисленные категории.
# flat_type - Своб. планировка.
test_correct = test.drop(test[test['flat_type'] == 'Своб. планировка'].index)
# rooms - Своб. планировка, 7, 8, 9.
test_correct.drop(test_correct[(test_correct['rooms'] == 'Своб. планировка') | (test_correct['rooms'] == '7') | \
                   (test_correct['rooms'] == '8') | (test_correct['rooms'] == '9')].index, inplace=True)
# floors - 26 - 35.
test_correct.drop(test_correct[(test_correct['floors'] == '26') | (test_correct['floors'] == '27') | \
                   (test_correct['floors'] == '28') | (test_correct['floors'] == '29') | \
                   (test_correct['floors'] == '30') | (test_correct['floors'] == '31') | \
                   (test_correct['floors'] == '32') | (test_correct['floors'] == '33') | \
                   (test_correct['floors'] == '34') | (test_correct['floors'] == '35')].index, inplace=True)
# build_matireal - блоки, дерево.
test_correct.drop(test_correct[(test_correct['build_matireal'] == 'Блоки') | \
                           (test_correct['build_matireal'] == 'Дерево')].index, inplace=True)
# total_floors - 1, 30 - 36.
test_correct.drop(test_correct[(test_correct['total_floors'] == '1') | (test_correct['total_floors'] == '30') | \
                   (test_correct['total_floors'] == '31') | (test_correct['total_floors'] == '32') | \
                   (test_correct['total_floors'] == '33') | (test_correct['total_floors'] == '34') | \
                   (test_correct['total_floors'] == '35') | (test_correct['total_floors'] == '36')].index, inplace=True)
# metro_station - 5км, Пискаревка.
test_correct.drop(test_correct[(test_correct['metro_station'] == '5 км') | \
                           (test_correct['metro_station'] == 'Пискарёвка')].index, inplace=True)

## < 400 m2

In [29]:
comparison_report_cat_cor_400m2 = sv.compare([train_correct[(train_correct['square']<401) & (train_correct['price']<100000000)],'Train'], \
                                             [test_correct[(test_correct['square']<401) & (test_correct['price']<100000000)],'Test'], \
                                             target_feat='price')

                                             |          | [  0%]   00:00 -> (? left)

In [30]:
# Сохраняем результат в виде HTML.
comparison_report_cat_cor_400m2.show_html(filepath='NEW_FEATURES_400M2_SWEETVIZ_REPORT.html')

Report NEW_FEATURES_400M2_SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## < 130 m2

In [31]:
comparison_report_cat_cor_130m2 = sv.compare([train_correct[(train_correct['square']<131) & (train_correct['price']<100000000)],'Train'], \
                                             [test_correct[(test_correct['square']<131) & (test_correct['price']<100000000)],'Test'], \
                                             target_feat='price')

                                             |          | [  0%]   00:00 -> (? left)

In [32]:
# Сохраняем результат в виде HTML.
comparison_report_cat_cor_130m2.show_html(filepath='NEW_FEATURES_130M2_SWEETVIZ_REPORT.html')

Report NEW_FEATURES_130M2_SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
